# Worksheet 08

Name:  MANUSHI MUNSHI

UID: U25855816 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.759565668458676, 5.188732583288381, 4.956874501290525, 4.529221172051075, 4.490593714301451, 5.8997325740829325, 4.712817252257899, 3.998526724000028, 5.438430237491119, 4.979678376294689]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal( 8 , 1 ,10 ).tolist()
print(s2)

[6.980086382730489, 8.140384231825958, 7.93719814753181, 8.826948040693685, 7.732754421618158, 8.029543005852384, 10.375825239936157, 7.470955941674377, 7.696960054128352, 7.268919135111995]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.268919135111995, 4.979678376294689, 7.696960054128352, 5.438430237491119, 3.998526724000028, 7.470955941674377, 4.712817252257899, 5.8997325740829325, 10.375825239936157, 8.029543005852384]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Number of components
- Mean (5 and 8 respectively) 
- Variance (1)
- Prior probability = 1/2

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[0])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.979678376294689, 5.438430237491119, 3.998526724000028, 4.712817252257899, 5.8997325740829325]
[7.268919135111995, 7.696960054128352, 7.470955941674377, 10.375825239936157, 8.029543005852384]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.005837032825333,  mean_2 = 8.168440675340651
var_1 = 0.4174810089568476,  var_2 = 11.284013221614238


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)
prob_x = []    # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_i = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_s[0]) / prob_x_i
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_s[1]) / prob_x_i
    prob_s1_x.append(prob_s1_x_i)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()


point =  7.268919135111995
probability of observing that point if it came from cluster 0 =  3.975392164932136e-07
probability of observing that point if it came from cluster 1 =  0.03524248884950159
point =  4.979678376294689
probability of observing that point if it came from cluster 0 =  0.9537198183140015
probability of observing that point if it came from cluster 1 =  0.03397078536384622
point =  7.696960054128352
probability of observing that point if it came from cluster 0 =  9.064323830840508e-10
probability of observing that point if it came from cluster 1 =  0.03532379685910856
point =  5.438430237491119
probability of observing that point if it came from cluster 0 =  0.5586254875293367
probability of observing that point if it came from cluster 1 =  0.03433493114000733
point =  3.998526724000028
probability of observing that point if it came from cluster 0 =  0.052011296834063504
probability of observing that point if it came from cluster 1 =  0.03302118184983493
point =  7.4

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * ((x[1] - mean[0])**2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x),
       sum([x[0] * ((x[1] - mean[1])**2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]


print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 5.039940557850209,  mean_2 = 7.713089960855763
var_1 = 0.3473212867488901,  var_2 = 2.525720585581666


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_s0_x = []  # P(S_0 | X_i)
prob_s1_x = []  # P(S_1 | X_i)

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))

    # P(S_0 | X_i) = P(X_i | S_0)P(S_0) / P(X_i)
    prob_s0_x_i = (pdf_i[0] * prob_c[0]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s0_x.append(prob_s0_x_i)

    # P(S_1 | X_i) = P(X_i | S_1)P(S_1) / P(X_i)
    prob_s1_x_i = (pdf_i[1] * prob_c[1]) / (pdf_i[0] * prob_c[0] + pdf_i[1] * prob_c[1])
    prob_s1_x.append(prob_s1_x_i)

# Print updated posterior probabilities
probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_0 = " + str(p[1]))
    print("Probability of coming from S_1 = " + str(p[2]))
    print()


7.268919135111995
Probability of coming from S_0 = 6.122513066346302e-09
Probability of coming from S_1 = 0.999999993877487

4.979678376294689
Probability of coming from S_0 = 0.9035034797447047
Probability of coming from S_1 = 0.09649652025529537

7.696960054128352
Probability of coming from S_0 = 1.036395308769102e-12
Probability of coming from S_1 = 0.9999999999989636

5.438430237491119
Probability of coming from S_0 = 0.804329596090336
Probability of coming from S_1 = 0.1956704039096639

3.998526724000028
Probability of coming from S_0 = 0.1483545213861207
Probability of coming from S_1 = 0.8516454786138793

7.470955941674377
Probability of coming from S_0 = 1.2230751129948568e-10
Probability of coming from S_1 = 0.9999999998776925

4.712817252257899
Probability of coming from S_0 = 0.8730517377155352
Probability of coming from S_1 = 0.1269482622844647

5.8997325740829325
Probability of coming from S_0 = 0.24230336689553955
Probability of coming from S_1 = 0.7576966331044604

10.37

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
hard_assignments = []  # List to store hard cluster assignments

for p_s0, p_s1 in zip(prob_s0_x, prob_s1_x):
    if p_s0 > p_s1:
        hard_assignments.append(0)  # Assign to cluster 0
    else:
        hard_assignments.append(1)  # Assign to cluster 1

# Print the hard assignments
for i, assignment in enumerate(hard_assignments):
    print(f"Data point {i} belongs to Cluster {assignment}")


Data point 0 belongs to Cluster 1
Data point 1 belongs to Cluster 0
Data point 2 belongs to Cluster 1
Data point 3 belongs to Cluster 0
Data point 4 belongs to Cluster 1
Data point 5 belongs to Cluster 1
Data point 6 belongs to Cluster 0
Data point 7 belongs to Cluster 1
Data point 8 belongs to Cluster 1
Data point 9 belongs to Cluster 1
